In [3]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from pathlib import Path
from torchsummary import summary
print("PyTorch:", torch.__version__)

PyTorch: 2.9.1+cpu


In [2]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
SEED = 42
import numpy as np, random, torch
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [5]:
# Set path to your data folder (relative to notebook). Adjust if needed.
DATA_DIR = "data"  # change only if your folder is elsewhere

# Find class folders
classes = sorted([d.name for d in Path(DATA_DIR).iterdir() if d.is_dir()])
print("Found classes:", classes)

# Prefer these two if available (case-sensitive)
preferred = []
if "Straight" in classes: preferred.append("Straight")
if "curly" in classes: preferred.append("curly")

# Fallback: pick first two
if len(preferred) < 2:
    if len(classes) >= 2:
        chosen = classes[:2]
    else:
        raise RuntimeError("Not enough class folders found in data/")
else:
    chosen = preferred[:2]

print("Using classes for binary classification:", chosen)


Found classes: ['Straight', 'Wavy', 'curly', 'dreadlocks', 'kinky']
Using classes for binary classification: ['Straight', 'curly']


In [6]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200,200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((200,200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# Use ImageFolder but restrict to the two chosen classes
full_dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
# Map class index -> class name
all_classes = full_dataset.classes
print("ImageFolder classes:", all_classes)

# Get indices of samples belonging to chosen classes
chosen_indices = [i for i,(path,label) in enumerate(full_dataset.samples) if all_classes[label] in chosen]
if len(chosen_indices) == 0:
    raise RuntimeError("No images found for chosen classes. Check DATA_DIR and class names.")

# Build a Subset dataset and remap labels to 0/1
subset = Subset(full_dataset, chosen_indices)

# To remap labels to 0/1 we create a small wrapper dataset
from torch.utils.data import Dataset
class BinarySubset(Dataset):
    def __init__(self, subset, chosen_names, transform=None):
        self.subset = subset
        self.chosen = chosen_names
        self.transform = transform
        # capture mapping from original class idx to class name
        self.orig_classes = full_dataset.classes

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        img, orig_label = self.subset[idx]  # orig_label is original class index
        orig_name = self.orig_classes[orig_label]
        new_label = float(self.chosen.index(orig_name))  # 0.0 or 1.0
        return img, torch.tensor(new_label, dtype=torch.float32)

# Create train/validation split (80/20)
n = len(subset)
indices = list(range(n))
random.shuffle(indices)
split = int(0.8 * n)
train_idx, val_idx = indices[:split], indices[split:]

train_dataset = Subset(subset, train_idx)
val_dataset = Subset(subset, val_idx)

train_dataset = BinarySubset(train_dataset, chosen, transform=None)
validation_dataset = BinarySubset(val_dataset, chosen, transform=None)

print("Train size:", len(train_dataset), "Validation size:", len(validation_dataset))


ImageFolder classes: ['Straight', 'Wavy', 'curly', 'dreadlocks', 'kinky']
Train size: 800 Validation size: 201


In [7]:
batch_size = 20

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)


In [8]:
class HairNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)  # no padding, stride=1 default
        self.pool = nn.MaxPool2d(2,2)
        # We'll create fc1 dynamically after knowing conv output size
        self.fc1 = None
        self.fc2 = nn.Linear(64, 1)  # placeholder; we'll replace if needed

    def _get_conv_output(self, shape=(3,200,200)):
        bs = 1
        input = torch.zeros(bs, *shape)
        out = F.relu(self.conv1(input))
        out = self.pool(out)
        return int(np.prod(out.size()[1:]))

    def build_fc(self):
        conv_out_size = self._get_conv_output()
        self.fc1 = nn.Linear(conv_out_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # NOTE: we'll use BCEWithLogitsLoss -> no sigmoid here
        return x

model = HairNet().to(device)
model.build_fc()
print(model)
# optional: show summary
try:
    summary(model, input_size=(3,200,200))
except:
    pass


HairNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


In [9]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)


In [10]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).unsqueeze(1)  # shape (B,1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}  Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.4f}  ValLoss: {val_epoch_loss:.4f}  ValAcc: {val_epoch_acc:.4f}")


Epoch 1/10  Loss: 0.6391  Acc: 0.6300  ValLoss: 0.6074  ValAcc: 0.6368
Epoch 2/10  Loss: 0.5814  Acc: 0.6737  ValLoss: 0.5247  ValAcc: 0.7313
Epoch 3/10  Loss: 0.5571  Acc: 0.7025  ValLoss: 0.4970  ValAcc: 0.7811
Epoch 4/10  Loss: 0.4971  Acc: 0.7350  ValLoss: 0.4925  ValAcc: 0.7612
Epoch 5/10  Loss: 0.4478  Acc: 0.7675  ValLoss: 0.5519  ValAcc: 0.7015
Epoch 6/10  Loss: 0.3686  Acc: 0.8200  ValLoss: 0.5430  ValAcc: 0.7114
Epoch 7/10  Loss: 0.3621  Acc: 0.8300  ValLoss: 0.5529  ValAcc: 0.6866
Epoch 8/10  Loss: 0.3653  Acc: 0.8512  ValLoss: 0.5998  ValAcc: 0.7313
Epoch 9/10  Loss: 0.2529  Acc: 0.9137  ValLoss: 0.4848  ValAcc: 0.8060
Epoch 10/10  Loss: 0.2273  Acc: 0.9125  ValLoss: 0.5660  ValAcc: 0.7761


In [11]:
import numpy as np
train_accs = np.array(history['acc'])
train_losses = np.array(history['loss'])

q3_median_acc = float(np.median(train_accs))
q4_std_loss = float(np.std(train_losses, ddof=0))  # population std to match typical choices

print("Training accuracies (per epoch):", train_accs)
print("Training losses (per epoch):", train_losses)
print(f"Q3 - median training acc: {q3_median_acc:.4f}")
print(f"Q4 - std training loss: {q4_std_loss:.4f}")


Training accuracies (per epoch): [0.63    0.67375 0.7025  0.735   0.7675  0.82    0.83    0.85125 0.91375
 0.9125 ]
Training losses (per epoch): [0.63905114 0.58138034 0.55706192 0.49714154 0.44783388 0.36857385
 0.36207833 0.36533775 0.25288861 0.22730693]
Q3 - median training acc: 0.7937
Q4 - std training loss: 0.1314


In [12]:
aug_train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9,1.0), ratio=(0.9,1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200,200)),  # ensure final size (some transforms may change it)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# Recreate full_dataset but with augmentation for training
full_dataset_aug = datasets.ImageFolder(DATA_DIR, transform=aug_train_transforms)
subset_aug = Subset(full_dataset_aug, chosen_indices)  # same indices selection as before

# re-split: we must use the same train/val split indices used earlier
train_subset_aug = Subset(subset_aug, train_idx)
val_subset_aug = Subset(subset_aug, val_idx)

train_dataset_aug = BinarySubset(train_subset_aug, chosen)
validation_dataset_aug = BinarySubset(val_subset_aug, chosen)

train_loader_aug = DataLoader(train_dataset_aug, batch_size=batch_size, shuffle=True, num_workers=0)
validation_loader_aug = DataLoader(validation_dataset_aug, batch_size=batch_size, shuffle=False, num_workers=0)

# Continue training for 10 more epochs
num_epochs_aug = 10
aug_history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    aug_history['loss'].append(epoch_loss)
    aug_history['acc'].append(epoch_acc)

    # validation (test) on the same validation dataset (with transform matching test_transforms)
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader_aug:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset_aug)
    val_epoch_acc = correct_val / total_val
    aug_history['val_loss'].append(val_epoch_loss)
    aug_history['val_acc'].append(val_epoch_acc)

    print(f"Aug Epoch {epoch+1}/{num_epochs_aug}  Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.4f}  ValLoss: {val_epoch_loss:.4f}  ValAcc: {val_epoch_acc:.4f}")


Aug Epoch 1/10  Loss: 0.6779  Acc: 0.6150  ValLoss: 0.5727  ValAcc: 0.6219
Aug Epoch 2/10  Loss: 0.5935  Acc: 0.6863  ValLoss: 0.5442  ValAcc: 0.6766
Aug Epoch 3/10  Loss: 0.5652  Acc: 0.7100  ValLoss: 0.5566  ValAcc: 0.6915
Aug Epoch 4/10  Loss: 0.5491  Acc: 0.7063  ValLoss: 0.5325  ValAcc: 0.7015
Aug Epoch 5/10  Loss: 0.5478  Acc: 0.7063  ValLoss: 0.5617  ValAcc: 0.7065
Aug Epoch 6/10  Loss: 0.5342  Acc: 0.7400  ValLoss: 0.5260  ValAcc: 0.7264
Aug Epoch 7/10  Loss: 0.5310  Acc: 0.7150  ValLoss: 0.4763  ValAcc: 0.7413
Aug Epoch 8/10  Loss: 0.5115  Acc: 0.7412  ValLoss: 0.5146  ValAcc: 0.7313
Aug Epoch 9/10  Loss: 0.4994  Acc: 0.7400  ValLoss: 0.4702  ValAcc: 0.7711
Aug Epoch 10/10  Loss: 0.5035  Acc: 0.7412  ValLoss: 0.4951  ValAcc: 0.7612


In [13]:
aug_val_losses = np.array(aug_history['val_loss'])
aug_val_accs = np.array(aug_history['val_acc'])

q5_mean_test_loss = float(np.mean(aug_val_losses))  # mean over all 10 augmentation epochs
# average of test accuracy for last 5 epochs (6..10) -> indices 5:10
q6_avg_last5_test_acc = float(np.mean(aug_val_accs[5:10]))

print("Augmentation-phase validation losses (10 epochs):", aug_val_losses)
print("Augmentation-phase validation accs (10 epochs):", aug_val_accs)
print(f"Q5 - mean test loss (10 aug epochs): {q5_mean_test_loss:.4f}")
print(f"Q6 - avg test acc (epochs 6-10): {q6_avg_last5_test_acc:.4f}")


Augmentation-phase validation losses (10 epochs): [0.57266151 0.54416775 0.55656575 0.53245739 0.56171739 0.52598743
 0.47634739 0.51464022 0.47017972 0.49509052]
Augmentation-phase validation accs (10 epochs): [0.62189055 0.67661692 0.69154229 0.70149254 0.70646766 0.72636816
 0.74129353 0.73134328 0.77114428 0.76119403]
Q5 - mean test loss (10 aug epochs): 0.5250
Q6 - avg test acc (epochs 6-10): 0.7463
